## Project: Automate Rejection Emails

Link to BIOSL task: https://bugs.opera.com/browse/BIOSL-302

Author: Linh Le / BI Intern

Requester: Jose Diaz

#### Input:
- Emails from bi.oslo.reporting@gmail.com

#### Output:
- Google sheet: https://docs.google.com/spreadsheets/d/1AfFa7lPHZ5_NgYNHAawcf4Wy5hGDpsY683T6UkBcb64/edit#gid=1200894280https://docs.google.com/spreadsheets/d/1AfFa7lPHZ5_NgYNHAawcf4Wy5hGDpsY683T6UkBcb64/edit#gid=1200894280


### 1. Read the current Google Sheet file into Python

In [22]:
import pandas as pd
import numpy as np
from datetime import date
import aiteam_tools.ai_tools.gsheets_api as gsheet
from apiclient import discovery

# Set up google sheets api
sheets = discovery.build("sheets", "v4").spreadsheets()
sheet_name = 'Rejection Emails'
gsID = '1AfFa7lPHZ5_NgYNHAawcf4Wy5hGDpsY683T6UkBcb64'

# Reading Google Sheet to Dataframe
current_email = gsheet.read_df(sheets, gsID, sheet_name, start_column="A", end_column="M", start_row="1", end_row="241") 
current_email.shape

# # Change the data type of "month" and "date" column for the purpose of concatenating data frames later
current_email['month'] = pd.to_datetime(current_email['month'])
current_email['date'] = pd.to_datetime(current_email['date'])

current_email.info()

(240, 13)

### 2. Extract all attachments from all emails

In [9]:
import mailbox
## (1) Define a function to extract attachments

number_of_attachment = []

def extractattachements(message):
    if message.get_content_maintype() == 'multipart':
        for part in message.walk():
            if part.get_content_maintype() == 'multipart': continue
            if part.get('Content-Disposition') is None: continue
            attachment_name = part.get_filename()
            number_of_attachment.append(attachment_name)
            attachment = open(attachment_name,'wb')
            attachment.write(part.get_payload(decode=True))
            attachment.close()

## (2) Apply the newly defined function to all emails in the mbox file:

# Reading the mbox file to Python
mbox = mailbox.mbox("Inbox 1005.mbox")

# Apply function to extract all attachments
for message in  mbox:
    extractattachements(message)
    
print("The number of attachments is: ", len(number_of_attachment))
            
### NOTE: All attachments are extracted and located in the current working directory on Pube


The number of attachments is:  252


### 3. Getting a list of attachment names from emails:

In [10]:
attachment_name = []

for message in mbox:
    if message.get_content_maintype() == 'multipart':
        for part in message.walk():
            if part.get_content_maintype() == 'multipart': continue
            if part.get('Content-Disposition') is None: continue
            filename = part.get_filename()
            attachment_name.append(filename)


### 4. Get the link to each specific attachment

In [11]:
import os
import re as _re
import json
import urllib
import urllib.parse
from apiclient import discovery
from googleapiclient.http import MediaFileUpload

service = discovery.build('drive', 'v3')

# Create folder + spredsheet in shared drive: Distribution Team/Tasks 
project = "email rejection "
task = "automation"
template = 'general'
create_drivefolder = True
attachment_name

# (1) Firstly, we create a parent Drive folder

bi_folder = '19C-3tKg0iPFIE7_-X3Fu9ZrhDlVHE3pN' # Folder of Email Rejection Automation Project

body = {"parents":[bi_folder], "name":"{}".format(project), "mimeType":"application/vnd.google-apps.folder"}
folder = service.files().create(body = body, supportsAllDrives=True).execute()
folder_id = folder['id']

# (2) Then, we upload the attachments extracted from defined function above to the drive folder

sheet_list = []

for file in attachment_name:
    media = MediaFileUpload(file, mimetype='application/pdf')
    
    body = {"parents":[folder_id], "name":"{} {}".format(file, task), "mimeType":"application/pdf"}
    the_sheet = service.files().create(body = body, media_body = media, supportsAllDrives=True).execute()
    sheet_id = the_sheet['id']
    sheet_list.append(sheet_id)
    

print('https://drive.google.com/drive/u/0/folders/' + folder_id)

# Create a list containing link to attachment
attachment_link = []

for sheet in sheet_list:
    att_link = 'https://drive.google.com/file/d/' + sheet
    attachment_link.append(att_link)


https://drive.google.com/drive/u/0/folders/187aaBMr886agMRXiQqevEmbSdHfhB_dw


### 5. Create a dataframe getting desired information from emails

In [12]:
import pandas as pd
import numpy as np

# (1) CREATING DATAFRAME FROM EMAIL

# Open the downloaded email (mbox file) to Python
with open(r"Inbox 1005.mbox", "r") as file:
    lines = file.read().split("\n")
    
# Creating a dictionary containing content we want to scrape from an email:
kw1 = "Email Subject"
kw2 = "Email Date"
kw3 = "Email Sender"
kw4 = "Email Recipient"
kw5 = "Error code"
kw6 = "Error details"
kw7 = "Email Part"

kw_search = [kw1, kw2, kw3, kw4, kw5, kw6, kw7]

email_subject = []
email_date = []
email_sender = []
email_recipient = [] 
error_code = []
error_detail = []
attachment_name = []


for line in lines:
    if kw_search[0] in line:
        email_subject.append(" ".join(line.split()[2:-1]))
        
    if kw_search[1] in line:
        email_date.append(" ".join(line.split()[2:-1]))
        
    if kw_search[2] in line:
        email_sender.append(" ".join(line.split()[2:-1]))
        
    if kw_search[3] in line:
        email_recipient.append(" ".join(line.split()[2:-1]))
        
    if kw_search[4] in line:
        error_code.append(" ".join(line.split()[2:]))
    
    if kw_search[5] in line:
        error_detail.append(" ".join(line.split()[2:-1]))
        
    if kw_search[6] in line:
        attachment_name.append(" ".join(line.split()[2:-1]))
        
# Removing HTML code <br> from "error code" column
error_code_df = []

for error in error_code:
    error_code_df.append(error[:-5])
        
    
# Create a dictionary
my_dict = {
    "email date" : email_date,
    "email subject" : email_subject,
    "email sender" : email_sender,
    "email recipient" : email_recipient,
    "error code" : error_code_df,
    "error details": error_detail,
    "attachment name" : attachment_name
}

# Convert the dictionary into a dataframe
inbox = pd.DataFrame(my_dict)

    
### (2) DATA CLEANING

# Add the "link to attachment" column 
inbox['link to attachment'] = attachment_link

# Add a "date" column 
date_col = []
for date in inbox['email date']:
    date_col.append(date[0:10])
    
inbox['date'] = date_col

# Change the type of "date" column to datetime type
inbox['date'] = pd.to_datetime(inbox['date'])

# Add the "month" column 
inbox['month'] = inbox['date'].dt.strftime("%Y-%m")

# Change the type of "month" column to datetime type
inbox['month'] = pd.to_datetime(inbox['month'])

# Fixing the timestamp column:
timestamp_col = []
for timestamp in inbox['email date']:
    timestamp_col.append(timestamp[0:19].replace("T", " "))
    
inbox['timestamp'] = timestamp_col 
del inbox['email date']


# Moving the "month" column to the first position
first_col = inbox.pop('month')
inbox.insert(0, "month", first_col)

# Moving the "date" column to the second position
sec_col = inbox.pop('date')
inbox.insert(1, "date", sec_col)

# Moving the "timestamp" column to the third position
third_col = inbox.pop('timestamp')
inbox.insert(2, "timestamp", third_col)

# Converting "timestamp" to string type
inbox['timestamp'] = inbox['timestamp'].astype('string')

# Sort the data frame by "timestamp"
inbox = inbox.sort_values(['timestamp'], ascending = True)

print(inbox.shape)
inbox.head(2)

(252, 10)


,month,date,timestamp,email subject,email sender,email recipient,error code,error details,attachment name,link to attachment
140,2023-02-01,2023-02-14,2023-02-14 00:52:22,"PANDA SECURITY, S.L.U.: Invoice VPI23000064",payables-no@opera.com,payables-no@opera.com,BN-DGT10501,Target validation failed: [EUGEN-T10-R037]-An ...,Mail Content,https://drive.google.com/file/d/1cP5QwMS7A76oV...
144,2023-02-01,2023-02-14,2023-02-14 09:49:19,Re: Status of my account and request for,not_reply@bscs.basware.com,payables-no@opera.com,BN-DGT13008,Incorrect document type/material + Reminder of...,Mail Content,https://drive.google.com/file/d/1MeCbgg5ZUigQh...
87,2023-02-01,2023-02-14,2023-02-14 14:06:21,Faktura,mjanta@opera.com,faktury-wro@opera.com,BN-DGT10501,Target validation failed: [BII2-T10-R004]-An i...,Mail Content,https://drive.google.com/file/d/1nLQut1IpjoJ2-...
145,2023-02-01,2023-02-14,2023-02-14 16:43:11,FW: Bank account,ldoyle@opera.com,sergeyk@opera.com,BN-DGT13050,Mandatory Field could not be interpreted,Mail Content,https://drive.google.com/file/d/1fKJPnjpkL6b7Q...
88,2023-02-01,2023-02-15,2023-02-15 03:33:07,payment requestGeoEdge service,shuxianl@opera.com,payables-no@opera.com,BN-DGT10501,Target validation failed: [BII2-T10-R003]-An i...,Mail Content,https://drive.google.com/file/d/1OfzbLQp5xCVBf...


### 6. Completing the updated dataframe with new emails appended

In [42]:
# Adding a "string" column to current_email df
current_email['temp_col'] = current_email['timestamp'] + ' '+ current_email['email subject'] + ' '+ current_email['email sender'] + ' ' + current_email['email recipient'] + ' '+ current_email['error code'] + ' '+ current_email['error details']
print(current_email.shape)                          

# Adding a new string column to inbox df
inbox['temp_col'] = inbox['timestamp'] + ' '+ inbox['email subject'] + ' '+ inbox['email sender'] + ' ' + inbox['email recipient'] + ' '+ inbox['error code'] + ' '+ inbox['error details']
print(inbox.shape)

# Find the number of duplicates and unique rows in two df
print(inbox['temp_col'].isin(current_email['temp_col']).value_counts())

# Get the new dataframe that only contains the new/unique rows
df_diff = inbox.loc[~inbox['temp_col'].isin(current_email['temp_col'])]
print("The number of new rows in the new data frame: ", df_diff.shape)

# Drop "temp_col" in both "current_email" and "df_diff" dataframe
del df_diff['temp_col']
del current_email['temp_col']

# Add 3 columns to the df_diff so that we could concatenate "current_email" df and "df_diff" df later
df_diff['Verified by'] = np.nan
df_diff['Comment'] = np.nan
df_diff['Status'] = np.nan

# Now, we concatenate "current_email" and "df_diff" together
data = [current_email,df_diff]
update_df = pd.concat(data)

print("Shape of the updated dataframe: ", update_df.shape)

(240, 14)
(252, 11)
True     240
False     12
Name: temp_col, dtype: int64
The number of new rows in the new data frame:  (12, 11)


### 5. Upload the dataframe to Google Sheet file

In [58]:
from datetime import date
import aiteam_tools.ai_tools.gsheets_api as gsheet
from apiclient import discovery


# Set up google sheets api
sheets = discovery.build("sheets", "v4").spreadsheets()
gsID = "1AfFa7lPHZ5_NgYNHAawcf4Wy5hGDpsY683T6UkBcb64"
sheet = "Rejection Emails"

# Insert the actual data
gsheet.upload_df(update_df, sheets, gsID, sheet, "A", 1)